# 모델 설계도

#### 1. 2017 ~ 2019년의 3년간의 자료가 있는 종목은 2549 종목

#### 2. 그 중에 3년치 데이터(733 obs)가 온전하게 있는 경우는 2324 종목

#### 3. Shift를 통하여 shitf_-5 ~ shift_-9의 X를 생성

#### 4. 그 종목들 중 10%에 대하여 Test로 분리

#### 5. 90%의 Train 데이터 중 다시 90 : 10 으로 Train과 Test를 랜덤으로 분리

#### 6. Random Forest 모형을 적용

#### 7. 10%의 Test 종목에 대하여 평균 RMSE 계산

In [ ]:
# Data Loading
import pandas as pd
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# 각 종목별 obs 개수
print("*"*22)
print("* 각 종목별 obs 개수 *")
print("*"*22)
print(data["code"].value_counts())

print("\n\n")

# 3년간의 데이터가 완전한 종목 코드
print("*"*42)
print("* 3년간 데이터가 완전한 종목 코드 733obs *")
print("*"*42)
FULL_DATA_CODE = data["code"].value_counts().index[0:2323]
print(FULL_DATA_CODE)

print("\n\n")

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("*"*50)
print("* 3년간의 데이터가 완전한 종목만 추출한 데이터 *")
print("*"*50)
data = data[data["code"].isin(FULL_DATA_CODE)]
print(data)

In [32]:
TEMP = pd.DataFrame()

for i in [12320] : 
    for obs in data.index : 
        if data.iloc[obs]["code"] == i : 
            TEMP = TEMP.append(data.iloc[obs])

KeyboardInterrupt: 